In [1]:
%matplotlib inline

# to set the path for importing HADES modules
import sys
sys.path.append("../source/")

import numpy as np
import matplotlib.pyplot as plt

import LatLongUTMconversion
from hades_input import hades_input
from hades_location import hades_location
import hades_utils

# Ridgecrest example to demonstrate how HADES works

HADES works via the principle of inter-event distance estimation.


[insert image here]




The data used for this tutorial is the _Ridgecrest 2019_ afterschock sequence after the Mw 6.4 earthquake, measured at two stations: ***WBS*** and ***WMF***, at a distance of () km and () km. The data is taken from: [_REFERENCE_]

In [2]:
# First we load our data
data_path = 'data/'                            
input_file = 'inputs/ridgecrest_gji.dat'       # This contains the event times and arrival time picks
station_file = 'inputs/stations_ridge.txt'     # This contains the station locations

# And think of a filename to deposit our results in
out_file = '/outputs/ridgecrest_twostations.txt'

We need the P and S-wave velocities as inputs to estimate the inter-event distance. These are homogeneous velocities, but the velocity between events is not expected to vary very much.

In [8]:
# Homogeneous velocity estimates for the Ridgecrest example
Vp = 6000
Vs = Vp/1.8

# Stations: pick one or preferably two stations out of your station list. In this case
# we only have two stations, so that is easy
stations = ['WBS', 'WMF']

In [10]:
# Create an input object with data loaded 

hobj = hades_input(
    data_path = data_path,          # the general path to the data
    event_file = input_file,        # the input event file path
    station_file = station_file,    # the station file path
    sta_select = stations           # your two stations
)


['WBS', 'WMF']
{'WBS': [-58075.507727615535, -18408.043043394107, 0.0], 'WMF': [-31629.50772728643, 45782.95695989812, 0.0]}


### Now comes an important choice. 

>Depending on how many locations you have already and the quality/uncertainty on those locations, you can choose whether you want to use either:

- One master event: HADES finds the events relative to one another on basis of a pre-defined estimated aperture between the master event and thee other chosen events. And then locates absolutely on basis of this master event. It then rotates the cluster on basis of the measured (ts-tp)
- Four master events: HADES constructs a coordinate system on basis of these four events, does not need to rotate, and finds the absolute location relative to these four absolutelly located master events.
- .>4 master events: same story, absolute cluster and cluster shape are constrained by the amount of master events. If your master events are very uncertain, maybe not the best way to go because you could introduce a lot of bias.